<a href="https://colab.research.google.com/github/pathansaira7/Urdu/blob/master/LR_urdu_mrc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stanza
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
import stanza
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np

stanza.download('ur') # download Urdu model
ur_nlp = stanza.Pipeline(lang='ur', processors='tokenize,mwt,pos,lemma,depparse')

     |████████████████████████████████| 194kB 2.8MB/s 


2020-08-10 07:16:31 INFO: Downloading default packages for language: ur (Urdu)...
2020-08-10 07:16:43 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-08-10 07:16:43 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-08-10 07:16:43 INFO: Loading these models for language: ur (Urdu):
| Processor | Package |
-----------------------
| tokenize  | udtb    |
| pos       | udtb    |
| lemma     | udtb    |
| depparse  | udtb    |

2020-08-10 07:16:43 INFO: Use device: cpu
2020-08-10 07:16:43 INFO: Loading: tokenize
2020-08-10 07:16:43 INFO: Loading: pos
2020-08-10 07:16:44 INFO: Loading: lemma
2020-08-10 07:16:44 INFO: Loading: depparse
2020-08-10 07:16:45 INFO: Done loading processors!


In [ ]:
SYNTACTIC_RELATIONS= ["nsubj",
"obj",
"iobj",
"csubj",
"ccomp",
"xcomp",
"obl",
"vocative",
"expl",
"dislocated",
"advcl",
"advmod",
"discourse",
"aux",
"cop",
"mark",
"nmod",
"appos",
"nummod",
"acl",
"amod",
"det",
"clf",
"case",
"conj",
"cc",
"fixed",
"flat",
"compound",
"list",
"parataxis",
"orphan",
"goeswith",
"reparandum",
"punct",
"root",
"dep"]

In [ ]:

def get_synt_dep_graph(ur_doc):
    list_edges = []
    #print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')
    for sent in ur_doc.sentences:
        for word in sent.words:
            edge_from = word.text+"-"+str(word.id)
            edge_to = f'{sent.words[word.head-1].text+"-"+str(word.head) if word.head > 0 else "root"}'
            edge = (edge_from, edge_to)
            list_edges.append(edge)
        #print(list_edges)
    graph = nx.Graph(list_edges)
    #nx.draw(graph, with_labels=True, font_weight='bold')
    return graph

def get_synt_dep_relation(ur_doc, word_with_id):
    if word_with_id!='root':
      word_id = word_with_id.split('-')[1]
      ur_word_in_list = [word for sent in ur_doc.sentences for word in sent.words if word.id==word_id]
      if not ur_word_in_list:
          return 0
      ur_word = ur_word_in_list[0]
    else:
      ur_word = [word for sent in ur_doc.sentences for word in sent.words if word.head==0][0]
    return ur_word.deprel

def get_list_dep_path_relations(ur_doc, graph, source_word_with_id, target_word_with_id):
    #print('Dep-Rel between {} and {}: '.format(source_word_with_id, target_word_with_id))
    if source_word_with_id not in graph.nodes or target_word_with_id not in graph.nodes:
      return []
    shortest_path = nx.shortest_path(graph, source_word_with_id, target_word_with_id)
    list_dep_path_relations = []
    for word_with_id in shortest_path:
      dep_relation = get_synt_dep_relation(ur_doc, word_with_id)
      list_dep_path_relations.append(dep_relation)
    #print(list_dep_path_relations)
    return list_dep_path_relations


def get_dep_tree_features(graph, ur_pas, ur_qst, ur_ans):
    max_relations = 6   # 6 max relations between words
    max_features_per_tuple = 10   # 10 max dep features per (pas, qst, ans)
    list_features = []
    for i in range(max_relations):
      list_features.append([0]*max_features_per_tuple) # initialise all with 0 padding
    index_feature = 0
    pas_words = [word for sent in ur_pas.sentences for word in sent.words]
    pas_words_text = [word.text for word in pas_words]
    qst_words = [word for sent in ur_qst.sentences for word in sent.words]
    ans_words = [word for sent in ur_ans.sentences for word in sent.words]
    # get id of start of the answer in the passage
    id_start_answer = 0
    for i, pas_word in enumerate(pas_words):
        good = False
        if pas_word.text ==ans_words[0].text:
          good = True
          for j in range(len(ans_words)-1):
              if ans_words[j+1].text!=pas_words[i+j+1].text:
                  good=False
                  break
        if good:
          id_start_answer = pas_word.id
          break
    start_ans_word_with_id = ans_words[0].text + "-" + str(id_start_answer)
    # get dep tree features of common words in question and passage
    for qst_word in qst_words:
      if index_feature<5:
        if qst_word.text in pas_words_text:
            index = pas_words_text.index(qst_word.text)
            pas_word_with_id = pas_words[index].text + "-" + str(pas_words[index].id)
            # get dep tree feature
            list_relations = get_list_dep_path_relations(ur_pas, graph, pas_word_with_id, start_ans_word_with_id)
            i = 0
            while i<10 and i<len(list_relations):
              if list_relations[i] in SYNTACTIC_RELATIONS:
                list_features[index_feature][i] = SYNTACTIC_RELATIONS.index(list_relations[i])+1
              i = i+1
            index_feature += 1
    # Flatten the list
    list_features = [item for sublist in list_features for item in sublist]
    #print(list_features)
    return list_features

pas = ' اور امجد کون تھے  رشد اور امجد کون تھے'
qst = 'رشد اور امجد کون تھے'
ans = 'رشد اور امجد کون تھے'
ur_pas, ur_qst, ur_ans = ur_nlp(pas), ur_nlp(qst), ur_nlp(ans)
dep_graph = get_synt_dep_graph(ur_pas)
dep_tree_features = get_dep_tree_features(dep_graph, ur_pas, ur_qst, ur_ans)



In [ ]:
labeled_data_file = "/content/drive/My Drive/labeled_data.xlsx"
df = pd.read_excel(labeled_data_file, sheet_name='labeled_data',  header=0)
df.head(3)


,paragraph,question,answer,label
0,مارگریٹ اسٹاؤٹ اور کیری ایم اسٹیٹن نے حال ہی م...,مریم پارکر فولٹ کس لئے جانا جاتا ہے؟,تنظیمی نظریہ اور تنظیمی طرز عمل کے شعبوں میں س...,1
1,19 مارچ 2015 کو اعلان کیا گیا تھا کہ جزیرے کے ...,دولت مشترکہ نے نورفوک جزیرے کی حکومت کو کس جگہ...,ایک مقامی کونسل,1
2,اگرچہ کچھ ٹیموں نے کافی میدان اور حتی کہ مالی ...,سی بھیڑیے کس شہر میں منتقل ہوگئے؟,ٹورنٹو,1


In [ ]:
df['paragraph'][0]

'مارگریٹ اسٹاؤٹ اور کیری ایم اسٹیٹن نے حال ہی میں وہائٹ \u200b\u200bہیڈ اور مریم پارکر فولٹ کے باہمی اثر و رسوخ پر بھی تحریر کیا ہے ، جو تنظیمی نظریہ اور تنظیمی طرز عمل کے شعبوں میں سرخیل ہیں۔\xa0اسٹاؤٹ اور اسٹیٹن نے وہائٹ \u200b\u200bہیڈ اور فوللیٹ دونوں کو ایک ایسا عضو تناسل بانٹتے ہوئے دیکھا ہے جو "ایک رشتہ دارانہ عمل کے طور پر بننا سمجھتا ہے؛ فرق جیسا کہ تعلق ہے ، ابھی بھی انوکھا ہے as اور ہم آہنگ فرق بننے کا مقصد۔"\xa0اس روابط کا مزید تجزیہ اسٹوٹ اور جینین ایم محبت نے انٹیگریٹیو پروسیس میں کیا: فولٹین سوچ سے متعلق'

In [ ]:
from tqdm import tqdm_notebook
import pickle

list_all_dep_features = []
for i in tqdm_notebook(range(len(df['paragraph']))):
  par, qst, ans = df['paragraph'][i], df['question'][i], df['answer'][i]
  ur_pas, ur_qst, ur_ans = ur_nlp(par), ur_nlp(qst), ur_nlp(ans)
  dep_graph = get_synt_dep_graph(ur_pas)
  list_dep_features = get_dep_tree_features(dep_graph, ur_pas, ur_qst, ur_ans)
  list_all_dep_features.append(list_dep_features)
  if (i+1)%5000==0:
    features_to_add = list_all_dep_features[len(list_all_dep_features)-5000:]
    with open('/content/drive/My Drive/dep_features.pickle', 'rb') as f:
        curr_list_features = pickle.load(f)
    with open('/content/drive/My Drive/dep_features.pickle', 'wb') as f:
        curr_list_features.extend(features_to_add)
        pickle.dump(curr_list_features, f)
    print('Saved dep features of first {} data rows'.format(i+1))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Saved dep features of first 5000 data rows
Saved dep features of first 10000 data rows


True